In [8]:
import requests
from bs4 import BeautifulSoup

def get_rotowire_schedule(today=True):
    if today:
        url = "https://www.rotowire.com/baseball/daily-lineups.php"
    else:
        url = "https://www.rotowire.com/baseball/daily-lineups.php?date=tomorrow"

    response = requests.get(url)
    
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        lineup_boxes = soup.find_all("div", class_="lineup__box")
        game_times = soup.find_all("div", class_="lineup__time")
        return lineup_boxes, game_times
        #for box in lineup_boxes:
        #    content = box.get_text(strip=True)
        #    print(content)
    else:
        print(f"Failed to fetch webpage. Status code: {response.status_code}")


games, game_times = get_rotowire_schedule()


from importlib import reload
from src.simulation import Game as g
from src.simulation import Team as t
reload(g)
reload(t)

game = g.Game(games[0], game_times[0].get_text(strip=True) )

2024-05-25 15:14:20,816 - src.simulation.Game - INFO - Running init for TOR @ DET starting at 1:10 PM ET in Comerica Park
2024-05-25 15:14:20,817 - src.simulation.Game - INFO - Fitting hit classification model for Comerica Park
2024-05-25 15:14:30,825 - src.simulation.Game - INFO - Loading game state transition probs
2024-05-25 15:14:30,827 - src.simulation.Game - INFO - Fitting home team models
2024-05-25 15:14:30,869 - simulation.Pitcher - INFO - Starting init for Reese Olson
2024-05-25 15:14:35,888 - features.build_features - WARNING - low pitch count (17) for L SI
2024-05-25 15:14:46,134 - features.build_features - WARNING - low pitch count (28) for R CU
2024-05-25 15:14:47,506 - simulation.Pitcher - INFO - Reese Olson throws R with arsenal ['SI' 'FF' 'SL' 'CU' 'CH']
2024-05-25 15:14:47,507 - simulation.Pitcher - INFO - Init complete for Reese Olson
2024-05-25 15:14:47,542 - simulation.Batter - INFO - Starting init for Wenceel Perez
2024-05-25 15:14:51,962 - simulation.Batter - INF

In [65]:
from pprint import pprint

def check_for_ab_event():
    if game.game_state.balls > 3:
        return 'walk'
    elif game.game_state.strikes > 2:
        return 'K'
    return None
    
def simulate_pitch(fielding_team, batting_team):
    
    cur_batter = batting_team.batting_lineup[game.game_state.home_cur_batter]
    cur_pitcher = fielding_team.starting_pitcher
    
    cur_game_state = game.game_state.get_game_state()

    pitch_type, pitch_chars = cur_pitcher.generate_pitch(cur_game_state, cur_batter.stats, cur_pitcher.get_pitcher_state())
    pitch_outcome = cur_batter.generate_pitch_outcome(cur_game_state, pitch_chars)

    print(f'{cur_pitcher.name} threw a {pitch_type} and {cur_batter.name} got a {pitch_outcome}')

    cur_pitcher.cumulative_pitch_number += 1
    cur_pitcher.prev_pitch = pitch_type
    game.game_state.pitch_number += 1

    #process pitch outcome
    if pitch_outcome == 'hit_into_play':
        hit_chars = cur_batter.generate_hit_outcome()
        #game_state, fielding_team_name, batter_speed, hit_stats, fielding_oaa
        hit_class = game.classify_hit(game.game_state.get_game_state(),
                                      fielding_team.name,
                                      cur_batter.speed,
                                      cur_batter.stand,
                                      hit_chars,
                                      0 #TODO ADD FIELDING OAA
                                     )
        return hit_class
        
    elif pitch_outcome == 'strike':
        game.game_state.strikes += 1
    elif pitch_outcome == 'ball':
        game.game_state.balls += 1
    elif pitch_outcome == 'foul':
        if game.game_state.strikes < 2:
            game.game_state.strikes += 1
    elif pitch_outcome == 'hit_by_pitch':
        game.game_state.balls = 4

    return check_for_ab_event()

In [137]:
def process_event(event, cur_batter):

    cur_game_state = game.game_state.encode_cur_state()
    
    if event in ('K', 'walk'):
        new_state = game.deterministic_transitions[event][cur_state]
    else:
        new_state = game.sample_next_state(event, cur_batter.stand, cur_game_state)

    print('Current game state:')
    print(cur_game_state)
    print(f'{cur_batter.name} hit a {hit_class}')
    print('After game state:')
    print(game.game_state.encode_cur_state())
    game.game_state.process_state_change(new_state, cur_batter)


In [138]:
def simulate_ab(fielding_team, batting_team):
    game.game_state.reset_count()
    outcome = None
    while outcome is None:
        try:
            outcome = simulate_pitch(fielding_team, batting_team)
        except:
            print('Pitch sampling error')

    process_event(outcome, batting_team.batting_lineup[game.game_state.home_cur_batter])
    print(game.game_state.get_game_state())

simulate_ab(game.home_team, game.away_team)

Sampling remaining columns: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.31it/s]


Reese Olson threw a SI and Davis Schneider got a ball


Sampling remaining columns: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.05it/s]


Reese Olson threw a SI and Davis Schneider got a strike


Sampling remaining columns: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.77it/s]


Reese Olson threw a SL and Davis Schneider got a strike


Sampling remaining columns: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.94it/s]


Reese Olson threw a SI and Davis Schneider got a strike


AttributeError: 'Game' object has no attribute 'deterministic_transitions'

In [98]:
game.game_state.bases
game.game_state.encode_cur_state()

(3, (False, False, False))

In [94]:
game.game_state.get_game_state()

{'game_year': 2024,
 'strikes': 2,
 'balls': 4,
 'pitch_number': 10,
 'outs_when_up': 3,
 'on_1b': 0,
 'on_2b': 0,
 'on_3b': 0}

In [64]:
simulate_pitch(game.home_team, game.away_team)
if (event := check_for_ab_event()) is not None:
    print(f'{event} occured')
    #game.game_state.ab_change()
    
print(game.game_state.get_game_state())
print(game.game_state.encode_cur_state())

Sampling remaining columns: 100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.95it/s]

Reese Olson threw a SI and Davis Schneider got a hit_into_play
Current game state:
(0, (False, False, False))
Davis Schneider hit a single
After game state:
(0, (True, False, False))
{'game_year': 2024, 'strikes': 1, 'balls': 0, 'pitch_number': 3, 'outs_when_up': 0, 'on_1b': 1, 'on_2b': 0, 'on_3b': 0}
(0, (True, False, False))


In [106]:
import pickle
with open('game_state_t_probs.pkl', 'rb') as f:
    mp = pickle.load(f)

In [135]:
from itertools import product

def get_deterministic_transition_map():
    deterministic_transitions = {}
    deterministic_transitions['K'] = {}
    deterministic_transitions['walk'] = {}
    
    # Generate all permutations
    permutations = list(product(range(3), [(False, False, False), (True, False, False), (False, True, False), (False, False, True),
                                          (True, True, False), (False, True, True), (True, False, True), (True, True, True)]))
    for state in permutations:
        deterministic_transitions['K'][state] = (state[0] + 1, state[1], 0)
    
        first, second, third = state[1]
    
        new_first, new_second, new_third = 'batter', False, False
        runs_scored = 0
        if first and second and third:
            new_second = '1b'
            new_third = '2b'
            runs_scored = 1
        elif first and not second and not third:
            new_second = '1b'
        elif first and second:
            new_second = '1b'
            new_third = '2b'
        elif first and third:
            new_second = '1b'
            new_third = '3b'
        else:
            if second:
                new_second = '2b'
            if third:
                new_third = '3b'
    
        deterministic_transitions['walk'][state] = (state[0], (new_first, new_second, new_third), runs_scored)
    return deterministic_transitions

    
get_deterministic_transition_map()

{'K': {(0, (False, False, False)): (1, (False, False, False), 0),
  (0, (True, False, False)): (1, (True, False, False), 0),
  (0, (False, True, False)): (1, (False, True, False), 0),
  (0, (False, False, True)): (1, (False, False, True), 0),
  (0, (True, True, False)): (1, (True, True, False), 0),
  (0, (False, True, True)): (1, (False, True, True), 0),
  (0, (True, False, True)): (1, (True, False, True), 0),
  (0, (True, True, True)): (1, (True, True, True), 0),
  (1, (False, False, False)): (2, (False, False, False), 0),
  (1, (True, False, False)): (2, (True, False, False), 0),
  (1, (False, True, False)): (2, (False, True, False), 0),
  (1, (False, False, True)): (2, (False, False, True), 0),
  (1, (True, True, False)): (2, (True, True, False), 0),
  (1, (False, True, True)): (2, (False, True, True), 0),
  (1, (True, False, True)): (2, (True, False, True), 0),
  (1, (True, True, True)): (2, (True, True, True), 0),
  (2, (False, False, False)): (3, (False, False, False), 0),
  (2, 

In [126]:
mp['single']['L'][list(mp['single']['L'].keys())[2]]

{(1, ('batter', '1b', False), 0): 0.6493975903614457,
 (1, ('batter', False, '1b'), 0): 0.2927710843373494,
 (2, (False, 'batter', False), 0): 0.004417670682730923,
 (1, (False, 'batter', '1b'), 0): 0.014457831325301205,
 (1, (False, '1b', False), 0): 0.00642570281124498,
 (2, (False, False, False), 1): 0.0012048192771084338,
 (2, (False, False, '1b'), 0): 0.006827309236947791,
 (2, ('batter', False, False), 0): 0.011244979919678716,
 (1, ('batter', False, False), 1): 0.0008032128514056225,
 (1, (False, False, '1b'), 0): 0.004016064257028112,
 (1, ('batter', False, False), 0): 0.001606425702811245,
 (1, (False, False, 'batter'), 1): 0.0024096385542168677,
 (3, (False, False, False), 0): 0.0008032128514056225,
 (2, (False, '1b', False), 0): 0.00040160642570281126,
 (1, (False, False, False), 0): 0.00040160642570281126,
 (1, (False, 'batter', False), 1): 0.001606425702811245,
 (1, (False, False, False), 1): 0.00040160642570281126,
 (1, (False, False, False), 2): 0.00040160642570281126,
 